In [1]:
import pandas as pd

In [2]:
CITY = "madrid"

In [19]:
distance = pd.read_csv(
    f"../output/{CITY}/distance.csv",
    dtype={
        "stop_id": str,
    },
)
ac = pd.read_csv(
    f"../output/{CITY}/amenity_counts_in_accessibility.csv",
    dtype={
        "stop_id": str,
    },
)
pt_ac = pd.read_csv(
    f"../output/{CITY}/amenity_counts_in_public_transport_accessibility.csv",
    dtype={
        "stop_id": str,
    },
)
stop_geometries = pd.read_csv(
    f"../output/{CITY}/stop_geometries_from_walk.csv",
    dtype={
        "stop_id": str,
    },
)

stop_centralities = pd.read_csv(
    f"../data/stops/{CITY}/stops_with_centrality.csv",
    dtype={
        "stop_id": str,
    },
)
stop_centralities.drop(["Node"], axis=1, inplace=True)
stop_centralities.columns = [
    "eigenvector_centrality",
    "degree_centrality",
    "closeness_centrality",
    "betweenness_centrality",
    "stop_id",
    "cluster",
    "stop_lat",
    "stop_lon",
    "stop_name",
]
stop_centralities.dropna(subset=["stop_id"], inplace=True)

In [20]:
stop_centralities

,eigenvector_centrality,degree_centrality,closeness_centrality,betweenness_centrality,stop_id,cluster,stop_lat,stop_lon,stop_name
0,2.988838e-06,0.001158,0.061367,0.006125,5130,3,40.50730,-3.68606,Monasterio De Arlanza - Nuestra Señora De Valv...
1,2.988838e-06,0.001158,0.061367,0.006125,5166,3,40.50767,-3.68597,Monasterio De Arlanza - Nuestra Señora De Valv...
2,1.436234e-05,0.002703,0.065333,0.016070,3881,690,40.51320,-3.67918,Carretera De Alcobendas - Bercianos
3,1.436234e-05,0.002703,0.065333,0.016070,3882,690,40.51380,-3.67769,Carretera De Alcobendas - Bercianos
4,6.454423e-07,0.001158,0.057939,0.002703,1026,3386,40.50958,-3.69365,Monasterio Escorial - Monasterio Sobrado
...,...,...,...,...,...,...,...,...,...
4917,2.259148e-06,0.000772,0.052372,0.001926,5299,3438,40.40152,-3.56362,Mario Moreno Cantinflas - Victoria Kent
4918,1.089684e-05,0.000772,0.055251,0.002695,51003,3439,40.40507,-3.56039,Miguel Delibes - Mario Moreno Cantinflas
4919,2.341392e-06,0.000772,0.052374,0.001928,50007,3442,40.40161,-3.56102,Concejal V. Granizo - Victoria Kent
4920,3.758717e-08,0.000772,0.045281,0.000772,51007,3443,40.39769,-3.54685,Ferenc Puskas - Pilar Bellosillo


In [21]:
wk_amenity = ac.query("costing == 'walk' & range == 15").copy()
wk_amenity.drop(["costing", "range"], axis=1, inplace=True)
wk_amenity.columns = ["stop_id"] + [f"{i}_walk15" for i in wk_amenity.columns[1:]]
mm_amenity = pt_ac.copy()
mm_amenity.drop(["costing", "range"], axis=1, inplace=True)
mm_amenity.columns = ["stop_id"] + [f"{i}_multimodal" for i in mm_amenity.columns[1:]]

In [23]:
m = (
    stop_geometries.drop("geometry", axis=1)
    .merge(distance, on="stop_id")
    .merge(mm_amenity, on="stop_id")
    .merge(wk_amenity, on="stop_id")
    .merge(stop_centralities, on="stop_id")
)
m.to_csv(f"../output/{CITY}/merged.csv", index=False)